# Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import date

# Data Import & join
### Datasets: PALMS
First ensure that none of the files are duplicated by checking control sums.

In [2]:
start_year = 2015
start_month = 1
n_files = 81

df_temp = pd.read_csv("data/Region_Summary_PALMS_Report_2015_01.csv", index_col=0, encoding="ISO-8859-1")
sum_previous = df_temp.sum()

for i in range(n_files):
    month = (start_month + i) % 12 + 1
    month = str(month)
    if len(month) == 1:
        month = "0" + month
    else:
        pass
    year = start_year + ((start_month + i) // 12)
    
    df_temp = pd.read_csv(f"data/Region_Summary_PALMS_Report_{year}_{month}.csv", index_col=0, encoding="ISO-8859-1")
    sum_current = df_temp.sum()
    
    # If all of the column sums are the same, then sum of the boolean comparison on the left
    # will be equal to the number of columns seen on the right
    if (sum_previous == sum_current).sum() == sum_current.shape[0]:
        print("Duplicated readings")
        print(f"Current file: region-palms-report_{year}_{month}.csv")
#         break
    else:
        sum_previous = sum_current

The loop hasn't been broken for any of the instances which means that the files are not duplicate. At least they are not positioned month by month but it is even more unlikely that a duplicated file has been saved in a file where the month differs by more than one.

Just to double-check lets check the condition for the same file.

In [3]:
(sum_current == sum_current).sum() == sum_current.shape[0]

True

As expected - everything is working correctly. Lets import the files and concatenate them.

In [4]:
start_year = 2014
start_month = 12
n_files = 82

df_palms = pd.DataFrame()

for i in range(n_files):
    month = (start_month + i) % 12 + 1
    month = str(month)
    if len(month) == 1:
        month = "0" + month
    else:
        pass
    year = start_year + ((start_month + i) // 12)
#     print(f"region-palms-report_{year}_{month}.csv")
    
    df_temp = pd.read_csv(f"data/Region_Summary_PALMS_Report_{year}_{month}.csv", index_col=0, encoding="ISO-8859-1")
    df_temp["palms_date"] = date(year, int(month), 1)

    df_palms = pd.concat([df_palms, df_temp])

column_list = df_palms.columns.tolist()
column_list = column_list[-3:-1] + column_list[:-3] + [column_list[-1]]
df_palms = df_palms[column_list]

df_palms.reset_index(inplace=True, drop=True)
df_palms

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,RRO,V,1-2-1,TYFCB,CEU,palms_date
0,202,1,4,0,0,0,0,1,2,0,5,0,2,0,0,2015-01-01
1,1001,1,4,0,0,0,0,0,5,0,6,1,3,150,0,2015-01-01
2,1060,1,1,0,0,0,0,0,0,0,0,0,0,0,0,2015-01-01
3,702,1,4,0,0,0,0,0,4,0,0,0,2,0,0,2015-01-01
4,1634,1,3,0,0,1,0,1,0,0,5,0,1,0,0,2015-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38708,3005,32,2,1,1,0,0,1,2,2,1,0,0,100,0,2021-10-01
38709,3015,32,4,0,0,0,0,3,5,18,11,2,2,356,1,2021-10-01
38710,294,32,1,0,0,3,0,0,0,2,0,0,0,0,0,2021-10-01
38711,617,32,3,1,0,0,0,7,0,1,3,0,0,524,0,2021-10-01


### Dataset: database

In [5]:
df_database = pd.read_csv("data/database_data.csv", index_col=0, encoding="ISO-8859-1")
df_database.head(10)

,user_ID,chapter_ID,industry,sponsor_ID,join_date,renewal_date
0,1034,23,"Medical, Chiropractor",NaN,07/01/2004,12/01/2010
1,909,23,"Trades, Renovations-Remodeling",NaN,02/01/2008,12/01/2008
2,2062,23,"Food/Beverages, Chef",NaN,09/01/2010,12/01/2011
3,947,23,"Gifts, Gift Baskets",947.0,03/01/2007,04/07/2008
4,2311,23,"Mortgage, Mortgage Broker",NaN,07/01/2007,07/01/2008
5,1536,23,"Real Estate Services, Residential Real Estate ...",NaN,07/01/2007,04/01/2021
6,2486,23,"Insurance, Life,Health and Disability Insurance",NaN,07/01/2007,11/01/2009
7,2322,23,"Financial, Investment Advisor",NaN,07/01/2007,04/01/2009
8,878,23,"Health and Wellness, Massage Therapist",NaN,07/01/2007,07/01/2008
9,753,23,"Insurance, General-Motor Insurance",947.0,10/01/2007,10/01/2008


In [6]:
df_database["join_date"] = pd.to_datetime(df_database["join_date"], format='%m/%d/%Y', errors='coerce')
df_database["renewal_date"] = pd.to_datetime(df_database["renewal_date"], format='%m/%d/%Y', errors='coerce')
# df_database["sponsor_ID"] = pd.to_numeric(df_database["sponsor_ID"], errors='coerce', downcast='Int32')
df_database.head(10)

,user_ID,chapter_ID,industry,sponsor_ID,join_date,renewal_date
0,1034,23,"Medical, Chiropractor",NaN,2004-07-01,2010-12-01
1,909,23,"Trades, Renovations-Remodeling",NaN,2008-02-01,2008-12-01
2,2062,23,"Food/Beverages, Chef",NaN,2010-09-01,2011-12-01
3,947,23,"Gifts, Gift Baskets",947.0,2007-03-01,2008-04-07
4,2311,23,"Mortgage, Mortgage Broker",NaN,2007-07-01,2008-07-01
5,1536,23,"Real Estate Services, Residential Real Estate ...",NaN,2007-07-01,2021-04-01
6,2486,23,"Insurance, Life,Health and Disability Insurance",NaN,2007-07-01,2009-11-01
7,2322,23,"Financial, Investment Advisor",NaN,2007-07-01,2009-04-01
8,878,23,"Health and Wellness, Massage Therapist",NaN,2007-07-01,2008-07-01
9,753,23,"Insurance, General-Motor Insurance",947.0,2007-10-01,2008-10-01


### Dataset: dropped_members

In [7]:
df_dropped = pd.read_csv("data/dropped_members.csv", index_col=0, encoding="ISO-8859-1")
df_dropped.head(10)

,user_ID,chapter_ID,reason,drop_date
0,637.0,9,Left Company,11/03/2021
1,1185.0,23,Changed Job,10/22/2021
2,1225.0,14,Takes Too Much Time,10/13/2021
3,2138.0,15,Did Not Renew - No reason Given,11/01/2021
4,1610.0,15,Other Reason (see notes),10/12/2021
5,2574.0,12,Other Reason (see notes),10/28/2021
6,629.0,21,Changed Job,10/28/2021
7,722.0,13,Takes Too Much Time,10/22/2021
8,2580.0,23,Not Enough Referrals,09/23/2021
9,2336.0,6,Scheduling Conflicts,10/20/2021


In [8]:
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2223 entries, 0 to 2237
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   user_ID     2220 non-null   float64
 1   chapter_ID  2223 non-null   int64  
 2   reason      1355 non-null   object 
 3   drop_date   2223 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 86.8+ KB


Seems like there are some missing values in ```user_ID``` column, which is odd. There shouldn't be any missings values in this table.

In [9]:
df_dropped[df_dropped["user_ID"].isna()]

,user_ID,chapter_ID,reason,drop_date
884,NaN,11,Personal Reasons,01/10/2018
1053,NaN,31,Going back to School,03/28/2017
2147,NaN,9,NaN,01/06/2010


Disregards the records.

In [10]:
df_dropped.dropna(subset=["user_ID"], inplace=True)
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2220 entries, 0 to 2237
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   user_ID     2220 non-null   float64
 1   chapter_ID  2220 non-null   int64  
 2   reason      1353 non-null   object 
 3   drop_date   2220 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 86.7+ KB


In [11]:
df_dropped["drop_date"] = pd.to_datetime(df_dropped["drop_date"], format='%m/%d/%Y', errors='coerce')
df_dropped["user_ID"] = pd.to_numeric(df_dropped["user_ID"], downcast='integer')
df_dropped.head()

,user_ID,chapter_ID,reason,drop_date
0,637,9,Left Company,2021-11-03
1,1185,23,Changed Job,2021-10-22
2,1225,14,Takes Too Much Time,2021-10-13
3,2138,15,Did Not Renew - No reason Given,2021-11-01
4,1610,15,Other Reason (see notes),2021-10-12


## Ensure data Integrity
 1. Calculate correct membership length for chapter transfers/rejoins.
 2. Remove drop date from member transfers/rejoins **IF** there is a membership continuity.
 
 
 ### 1. Calculate correct membership length for chapter transfers/rejoins.

In [12]:
df_transfers = df_database.groupby("user_ID")[["chapter_ID"]].count().copy()
df_transfers.loc[df_transfers.index == 3089] = 0
df_transfers.rename({"chapter_ID": "re_joins"}, axis=1, inplace=True)

df_database = df_database.merge(df_transfers, how="left", on="user_ID")
df_database

,user_ID,chapter_ID,industry,sponsor_ID,join_date,renewal_date,re_joins
0,1034,23,"Medical, Chiropractor",NaN,2004-07-01,2010-12-01,1
1,909,23,"Trades, Renovations-Remodeling",NaN,2008-02-01,2008-12-01,2
2,2062,23,"Food/Beverages, Chef",NaN,2010-09-01,2011-12-01,2
3,947,23,"Gifts, Gift Baskets",947.0,2007-03-01,2008-04-07,3
4,2311,23,"Mortgage, Mortgage Broker",NaN,2007-07-01,2008-07-01,1
...,...,...,...,...,...,...,...
3562,983,0,"Finance & Insurance, Health Insurance",2219.0,2021-08-01,2022-08-01,1
3563,1052,0,"Finance & Insurance, Property & Casualty Insur...",2219.0,2021-06-01,2022-06-01,1
3564,210,0,"Real Estate Services, Home Inspection",NaN,2021-10-01,2022-10-01,1
3565,440,0,"Construction, HVAC - Heating & Air",1052.0,2021-11-01,2022-11-01,1


In [13]:
user_ID_list = df_palms["user_ID"].unique()
user_ID_list[:10]

array([ 202, 1001, 1060,  702, 1634, 2347, 2065, 2354, 2673, 1453],
      dtype=int64)

In [14]:
df_transfers = df_database.loc[(df_database["re_joins"] > 1) & (df_database["user_ID"].isin(user_ID_list))].copy()
df_transfers.sort_values("join_date", inplace=True)
df_transfers

,user_ID,chapter_ID,industry,sponsor_ID,join_date,renewal_date,re_joins
296,340,9,"Printing, Printer",NaN,1999-06-01,2013-09-01,2
500,1426,22,"Insurance, Property & Casualty Insurance",NaN,2000-08-01,2009-01-01,2
793,1048,20,"Marketing, Marketing Services",NaN,2005-11-01,2006-11-01,2
515,1766,12,"Real Estate, Real Estate Sales Representative ...",NaN,2006-02-01,2008-11-01,2
801,1761,20,"Mortgage, Mortgage Broker",743.0,2006-07-01,2007-08-27,2
...,...,...,...,...,...,...,...
1647,1102,10,"Legal & Accounting, Bookkeeping",NaN,2021-10-01,2022-10-01,2
787,2288,12,"Finance & Insurance, Group Benefits",1730.0,2021-10-01,2022-10-01,2
2472,1044,18,"Construction, Painter & Decorator",NaN,2021-10-01,2022-10-01,2
2650,7,19,"Construction, Electrician",NaN,2021-11-01,2023-02-01,5


In [15]:
df_temp = df_transfers.copy()
df_temp = df_temp.merge(df_dropped, how="left", on=["user_ID", "chapter_ID"])
df_temp.sort_values(["user_ID", "join_date", "drop_date"], ascending=True, inplace=True)
df_temp.drop_duplicates(["user_ID", "chapter_ID", "join_date"], keep="last", inplace=True)
df_temp.drop_duplicates(["user_ID", "chapter_ID", "drop_date"], keep="first", inplace=True)

df_temp.drop(["industry", "sponsor_ID", "renewal_date", "reason"], axis=1, inplace=True)

df2 = df_temp["drop_date"].isnull().groupby(df_temp["user_ID"]).sum().astype(int).reset_index(name='count')
df_temp = df_temp.merge(df2, on="user_ID")

for index, row in df_temp.loc[df_temp["count"] > 1].iterrows():
    cond1 = (df_temp["user_ID"] == row["user_ID"]) & (df_temp["chapter_ID"] == row["chapter_ID"]) & (df_temp["drop_date"].isna())
    cond2 = (df_palms["user_ID"] == row["user_ID"]) & (df_palms["chapter_ID"] == row["chapter_ID"])
    df_temp.loc[cond1, "drop_date"] = df_palms.loc[cond2, "palms_date"].max()

df_temp.dropna(subset=["drop_date"], inplace=True)
df_temp["drop_date"] = pd.to_datetime(df_temp["drop_date"])

df_temp["additional_months"] = (df_temp["drop_date"] - df_temp["join_date"]) / np.timedelta64(1, 'M')
df_temp["additional_months"] = df_temp["additional_months"].round().astype(int)

df_temp["additional_months"] = df_temp.groupby("user_ID")["additional_months"].shift(1, fill_value=0)
df_temp["additional_months_cumsum"] = df_temp.groupby(["user_ID"])["additional_months"].cumsum()

df_temp = df_temp.loc[df_temp["additional_months_cumsum"] != 0]
df_temp.drop(["re_joins", "count", "additional_months", "drop_date", "join_date"], axis=1, inplace=True)

df_temp.head(10)

,user_ID,chapter_ID,additional_months_cumsum
1,7,26,8
8,32,25,11
11,39,18,24
15,50,15,11
17,68,24,25
22,108,9,62
28,147,19,37
34,188,18,7
36,208,13,19
43,267,14,23


## Join data - create a master dataframe

In [16]:
df_master = df_palms.copy()
df_master = df_master.merge(df_database, how="left", on=["user_ID", "chapter_ID"])
df_master = df_master.merge(df_dropped, how="left", on=["user_ID", "chapter_ID"])

df_master.sample(10, random_state=13)

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,TYFCB,CEU,palms_date,industry,sponsor_ID,join_date,renewal_date,re_joins,reason,drop_date
27991,1518,18,4,0,0,0,0,1,0,0,...,0,4,2019-03-01,"Automotive, Automotive Sales and Service",3086.0,2018-06-01,2020-06-01,1,Other Reason (see notes),2020-01-07
30979,1314,6,4,0,0,0,0,1,12,0,...,21454,3,2019-08-01,"Real Estate Services, Commercial Real Estate",237.0,2016-04-01,2022-10-01,1,NaN,NaT
15072,250,23,5,0,0,0,0,1,6,0,...,7517,5,2017-06-01,"Real Estate, Real Estate Sales Representative ...",1012.0,2015-05-01,2018-05-01,1,Other Reason (see notes),2017-07-11
39192,2901,12,5,0,0,0,0,1,5,4,...,5425,25,2020-09-01,"Finance & Insurance, Financial Investments",1829.0,2018-12-01,2022-03-01,2,NaN,NaT
2948,2272,32,4,0,0,0,0,5,1,3,...,180,0,2015-07-01,"Photography, Photographer",NaN,2010-03-22,2017-02-01,1,Personal Reasons,2016-01-12
7274,3055,32,4,0,0,0,0,4,0,9,...,519,0,2016-04-01,"Advertising & Marketing, Printer",2212.0,2013-12-01,2021-06-01,2,Did Not Renew - No reason Given,2021-06-01
25424,1802,19,4,0,0,0,0,0,4,0,...,491,10,2018-11-01,"Employment Activities, Recruiter",NaN,2015-05-01,2022-05-01,1,NaN,NaT
1743,2861,12,4,0,0,0,0,1,3,1,...,545,0,2015-05-01,"Health and Wellness, Fitness Trainer",NaN,2013-03-01,2016-03-01,1,Other Reason (see notes),2016-03-01
30003,1739,19,4,0,0,0,0,0,3,5,...,0,11,2019-06-01,"Finance & Insurance, Property & Casualty Insur...",108.0,2018-12-01,2022-02-01,1,NaN,NaT
33433,1924,8,2,0,0,0,0,0,0,0,...,0,0,2019-12-01,"Consulting, Energy Consultant",1974.0,2018-12-01,2019-12-01,1,Other Reason (see notes),2019-12-05


# Data cleaning & aggregation
## Remove duplicates

In [17]:
df_master_clean = df_master.copy()

df_master_clean["palms_date"] = pd.to_datetime(df_master_clean["palms_date"], errors='coerce')
df_master_clean["renewal_date"] = pd.to_datetime(df_master_clean["renewal_date"], errors='coerce')

df_master_clean.shape[0]

47901

In [18]:
df_master_clean.sort_values(["palms_date", "join_date", "drop_date"], inplace=True)

df_master_clean.drop_duplicates(subset=df_master_clean.columns[:16].tolist() + ["join_date"], keep="last", inplace=True)
df_master_clean.drop_duplicates(subset=df_master_clean.columns[:16], inplace=True)
df_master_clean.shape[0]

38712

## Get relative renewal date for data aggregation

In [19]:
df_master_clean = df_master_clean.merge(df_temp, how="left", on=["user_ID", "chapter_ID"])
df_master_clean.loc[df_master_clean["additional_months_cumsum"].isna(), "additional_months_cumsum"] = 0
df_master_clean["additional_months_cumsum"] = df_master_clean["additional_months_cumsum"].astype(int)
df_master_clean.head()

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,CEU,palms_date,industry,sponsor_ID,join_date,renewal_date,re_joins,reason,drop_date,additional_months_cumsum
0,447,9,4,0,0,0,0,1,1,0,...,0,2015-01-01,"Finance & Insurance, Financial Investments",NaN,1998-05-01,2022-07-01,1,NaN,NaT,0
1,1366,9,4,0,0,0,0,4,0,0,...,0,2015-01-01,"Advertising & Marketing, Sign Company",NaN,2002-09-01,2022-06-01,1,NaN,NaT,0
2,1197,9,4,0,0,0,0,2,3,0,...,0,2015-01-01,"Coach, Business Coach",NaN,2006-11-01,2018-02-01,1,No Reason Entered,2018-01-11,0
3,1035,9,4,0,0,0,0,0,7,0,...,0,2015-01-01,"Insurance, Property & Casualty Insurance",1366.0,2007-07-01,2015-05-01,1,No Reason Entered,2015-06-04,0
4,1536,23,3,0,0,0,1,0,3,0,...,0,2015-01-01,"Real Estate Services, Residential Real Estate ...",NaN,2007-07-01,2021-04-01,2,Member Transferred BNI Chapters,2020-12-18,0


In [20]:
df_master_clean["year_of_membership"] = (df_master_clean["palms_date"] - df_master_clean["join_date"]) \
    / np.timedelta64(1, 'M') + df_master_clean["additional_months_cumsum"]
df_master_clean["year_of_membership"] = df_master_clean["year_of_membership"] // 12
df_master_clean["months_to_renewal"] = (df_master_clean["renewal_date"] - df_master_clean["palms_date"]) \
    / np.timedelta64(1, 'M')
df_master_clean["months_to_renewal"] = df_master_clean["months_to_renewal"].round().astype(int)
df_master_clean["years_to_renewal"] = (df_master_clean["months_to_renewal"] - 1) // 12
df_master_clean["months_to_renewal"] = df_master_clean["months_to_renewal"] % 12

# Substitute "months_to_renewal" value 0 with 12 for ease of aggregation later on
df_master_clean.loc[df_master_clean["months_to_renewal"] == 0, "months_to_renewal"] = 12

for index, row in df_master_clean.iterrows():
    df_master_clean.at[index, 'relative_renewal_date'] = row['renewal_date'] - pd.DateOffset(years=row['years_to_renewal'])

# Found a few instances where the "relative_renewal_date" would be subtracted incorrectly and lacking one day to be correct
df_master_clean.loc[df_master_clean["relative_renewal_date"].dt.day != 1, "relative_renewal_date"] = \
    df_master_clean.loc[df_master_clean["relative_renewal_date"].dt.day != 1, "relative_renewal_date"] + pd.DateOffset(days=1)
    
df_master_clean.drop(["years_to_renewal", "additional_months_cumsum"], axis=1, inplace=True)
df_master_clean.sample(10, random_state=23)

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,industry,sponsor_ID,join_date,renewal_date,re_joins,reason,drop_date,year_of_membership,months_to_renewal,relative_renewal_date
31267,2581,31,5,0,0,0,0,0,1,0,...,"Trades, Renovations - Interior",1363.0,2018-03-01,2021-03-01,1,Personal Reasons,2021-01-31,2.0,6,2021-03-01
9055,3042,9,3,0,0,0,0,0,1,0,...,"Administrative, Virtual Assistant",1197.0,2014-08-12,2018-08-01,2,Takes Too Much Time,2018-01-11,2.0,8,2017-08-01
4955,1129,12,4,0,0,0,0,3,0,2,...,"z(Archived Duplicate) Alternative Medicine, Na...",NaN,2015-12-01,2017-06-01,1,Takes Too Much Time,2017-03-28,0.0,4,2016-06-01
30619,2724,32,4,0,0,0,0,2,1,2,...,"Legal & Accounting, Business Law",2455.0,2012-03-01,2022-08-01,1,NaN,NaT,8.0,12,2021-08-01
28844,2598,25,2,0,0,0,0,0,0,0,...,"Cleaning, Dry Cleaning",3075.0,2019-07-01,2020-08-01,1,Not Right Fit with Company,2020-03-12,0.0,4,2020-08-01
29693,1902,24,4,0,0,0,0,1,0,0,...,"Finance & Insurance, Financial Investments",1635.0,2017-06-01,2022-05-01,1,NaN,NaT,3.0,11,2021-05-01
22808,2840,6,4,1,0,0,0,0,3,0,...,"Business, Business Broker",1314.0,2019-02-01,2020-04-01,1,No Reason Entered,2019-06-03,0.0,12,2020-04-01
3895,1413,12,2,0,0,0,0,1,3,0,...,"Business, Business Broker",1761.0,2011-02-01,2016-08-01,2,NaN,2016-08-01,4.0,8,2016-08-01
9592,1231,19,4,0,0,0,0,1,1,1,...,"Health and Wellness, Fitness Trainer",3047.0,2016-01-01,2020-07-01,1,Personal Reasons,2020-07-01,1.0,6,2017-07-01
14334,2222,15,3,0,0,0,0,3,5,8,...,"Car & Motorcycle, Auto/Car Repair",573.0,2017-07-01,2022-01-01,1,NaN,NaT,0.0,2,2018-01-01


In [21]:
df_master_clean.loc[(df_master_clean["user_ID"] == 2822) & (df_master_clean["relative_renewal_date"] == "2019-05-01")]

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,industry,sponsor_ID,join_date,renewal_date,re_joins,reason,drop_date,year_of_membership,months_to_renewal,relative_renewal_date
17219,2822,21,5,0,0,0,0,1,4,1,...,"z(Archived Duplicate) Alternative Medicine, Na...",NaN,2017-11-01,2020-05-01,1,Not Enough Referrals,2020-01-08,0.0,12,2019-05-01
17689,2822,21,4,0,0,0,0,0,9,3,...,"z(Archived Duplicate) Alternative Medicine, Na...",NaN,2017-11-01,2020-05-01,1,Not Enough Referrals,2020-01-08,0.0,11,2019-05-01
18149,2822,21,4,0,0,0,0,1,5,0,...,"z(Archived Duplicate) Alternative Medicine, Na...",NaN,2017-11-01,2020-05-01,1,Not Enough Referrals,2020-01-08,0.0,10,2019-05-01
18610,2822,21,4,0,1,0,0,1,3,2,...,"z(Archived Duplicate) Alternative Medicine, Na...",NaN,2017-11-01,2020-05-01,1,Not Enough Referrals,2020-01-08,0.0,9,2019-05-01
19080,2822,21,3,0,1,0,0,2,3,0,...,"z(Archived Duplicate) Alternative Medicine, Na...",NaN,2017-11-01,2020-05-01,1,Not Enough Referrals,2020-01-08,0.0,8,2019-05-01
19553,2822,21,4,0,0,0,0,1,2,0,...,"z(Archived Duplicate) Alternative Medicine, Na...",NaN,2017-11-01,2020-05-01,1,Not Enough Referrals,2020-01-08,0.0,7,2019-05-01
20036,2822,21,4,0,0,0,0,0,0,0,...,"z(Archived Duplicate) Alternative Medicine, Na...",NaN,2017-11-01,2020-05-01,1,Not Enough Referrals,2020-01-08,0.0,6,2019-05-01
20516,2822,21,3,0,0,0,0,0,3,0,...,"z(Archived Duplicate) Alternative Medicine, Na...",NaN,2017-11-01,2020-05-01,1,Not Enough Referrals,2020-01-08,1.0,5,2019-05-01
21021,2822,21,4,0,0,0,0,0,3,0,...,"z(Archived Duplicate) Alternative Medicine, Na...",NaN,2017-11-01,2020-05-01,1,Not Enough Referrals,2020-01-08,1.0,4,2019-05-01
21530,2822,21,4,0,0,0,0,2,4,1,...,"z(Archived Duplicate) Alternative Medicine, Na...",NaN,2017-11-01,2020-05-01,1,Not Enough Referrals,2020-01-08,1.0,3,2019-05-01


## Aggregate 9-months data
Ensure sure that each groupby sum is aggregated for 9 months, not less.

In [22]:
df_pre_agg = df_master_clean.copy()

df_pre_agg["control_count"] = 1
df_pre_agg = df_pre_agg.loc[df_pre_agg["months_to_renewal"] >= 4]

df_pre_agg.drop_duplicates(subset=df_pre_agg.columns[:16], inplace=True)

df_pre_agg.drop(["industry",
                 "sponsor_ID",
                 "re_joins",
                 "join_date",
                 "renewal_date",
                 "palms_date",
                 "drop_date",
                 "months_to_renewal"], axis=1, inplace=True)

df_pre_agg.sample(10, random_state=13)

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,RRO,V,1-2-1,TYFCB,CEU,reason,year_of_membership,relative_renewal_date,control_count
26831,2041,15,3,0,0,0,0,2,6,2,3,0,3,4075,2,NaN,0.0,2020-10-01,1
7599,1203,32,2,1,0,0,1,0,1,1,1,0,4,35,0,No Reason Entered,0.0,2017-07-01,1
21668,1462,9,0,0,0,4,0,0,0,0,5,0,0,906,0,"Company related (e.g. Changed Jobs, Left Compa...",1.0,2019-09-01,1
15418,295,17,4,0,0,0,0,1,1,4,5,0,4,0,0,No Reason Entered,6.0,2019-02-01,1
1981,2677,19,4,0,0,0,0,2,3,0,2,2,0,0,0,Other Reason (see notes),0.0,2016-05-01,1
32389,712,23,4,0,0,0,0,1,0,0,0,0,4,625,9,No Reason Entered,1.0,2021-10-01,1
10130,2111,25,2,2,0,0,0,1,2,0,0,0,0,4735,1,Changed Job,0.0,2017-06-01,1
1467,1361,10,3,0,0,0,1,5,0,1,1,1,0,1388,0,Other Reason (see notes),1.0,2016-01-01,1
35461,1906,11,4,0,0,0,0,5,2,2,0,0,5,1993,0,NaN,3.0,2022-01-01,1
18908,1464,11,1,2,0,0,0,0,0,0,0,0,2,0,0,Other Reason (see notes),2.0,2019-02-01,1


In [23]:
df_agg = df_pre_agg.copy()
df_agg = df_agg.groupby(["user_ID", "chapter_ID", "relative_renewal_date"]).sum()
df_agg.reset_index(drop=False, inplace=True)
df_agg = df_agg.loc[df_agg["control_count"] >= 9]
df_agg["year_of_membership"] = (df_agg["year_of_membership"]/9).round().astype(int)
df_agg.sample(10, random_state=13)

,user_ID,chapter_ID,relative_renewal_date,P,A,L,M,S,RGI,RGO,RRI,RRO,V,1-2-1,TYFCB,CEU,year_of_membership,control_count
2245,1622,32,2017-01-01,18,13,0,0,2,11,1,34,10,1,28,1087,0,0,9
23,10,19,2018-10-01,30,4,1,0,2,3,17,3,45,0,26,3648,57,1,9
2070,1505,18,2021-02-01,35,0,0,1,1,11,63,2,10,1,10,137945,33,2,9
3199,2285,15,2019-05-01,35,0,0,0,3,10,12,14,9,0,46,11719,41,1,9
2007,1460,25,2020-05-01,37,1,0,0,0,17,7,0,7,1,51,3478,53,0,9
2739,1952,32,2021-09-01,33,3,0,0,0,17,1,5,4,0,37,64919,8,0,9
1107,830,26,2019-11-01,32,2,2,0,1,8,19,0,6,2,23,39394,27,8,9
843,619,18,2018-08-01,33,0,0,0,4,14,4,3,20,2,36,5422,29,3,9
3771,2695,6,2020-07-01,32,3,0,0,3,21,19,16,70,1,42,42424,66,0,9
2188,1587,17,2017-04-01,34,3,1,0,1,7,13,13,13,3,25,7944,21,3,9


In [24]:
df_agg.shape

(2431, 18)

In [25]:
df_agg2 = df_pre_agg.copy()
df_agg2 = df_agg2.groupby(["user_ID", "relative_renewal_date"]).sum()
df_agg2.reset_index(drop=False, inplace=True)
df_agg2 = df_agg2.loc[df_agg2["control_count"] >= 9]
df_agg2["year_of_membership"] = (df_agg2["year_of_membership"]/9).round().astype(int)
df_agg2["chapter_ID"] = -1
df_agg2.sample(10, random_state=13)

,user_ID,relative_renewal_date,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,RRO,V,1-2-1,TYFCB,CEU,year_of_membership,control_count
754,573,2019-05-01,-1,38,0,0,0,0,13,55,13,68,3,57,152154,29,1,9
3936,2822,2019-05-01,-1,35,0,2,0,0,6,32,6,24,2,19,8720,50,0,9
1033,766,2020-02-01,-1,38,1,0,0,0,9,29,11,4,1,18,104530,44,6,9
194,142,2021-09-01,-1,37,0,0,0,0,3,41,39,80,11,49,5579,64,1,9
1332,1000,2020-05-01,-1,35,0,0,0,2,30,23,32,44,2,51,66134,66,6,9
2909,2073,2017-09-01,-1,22,2,0,8,5,3,20,1,6,5,54,28306,49,3,9
1411,1058,2021-12-01,-1,28,2,0,7,0,2,37,3,14,5,40,46689,64,5,9
960,712,2020-10-01,-1,34,1,0,0,1,6,4,5,2,5,41,5120,35,0,9
343,259,2018-11-01,-1,35,2,0,0,0,7,14,4,8,1,20,11174,5,7,9
2905,2071,2019-02-01,-1,31,2,3,0,2,8,11,3,18,2,22,43304,0,0,9


In [26]:
df_agg = pd.concat([df_agg, df_agg2])
del df_agg2
df_agg.sample(10, random_state=13)

,user_ID,chapter_ID,relative_renewal_date,P,A,L,M,S,RGI,RGO,RRI,RRO,V,1-2-1,TYFCB,CEU,year_of_membership,control_count
271,205,-1,2019-02-01,29,4,0,0,6,21,4,1,6,1,33,43687,6,3,9
832,616,-1,2019-05-01,31,4,0,0,1,15,51,30,49,14,32,38593,53,0,9
2776,1978,-1,2019-05-01,37,1,0,0,0,0,42,2,23,3,43,36458,44,1,9
1845,1363,-1,2021-09-01,34,1,0,0,2,5,16,2,3,4,32,42472,31,5,9
46,31,-1,2019-08-01,34,2,1,0,0,16,53,60,67,8,64,29839,87,1,9
1672,1245,19,2021-03-01,26,7,0,4,0,3,10,11,19,0,26,2209,29,6,9
558,442,-1,2020-11-01,32,0,0,2,0,9,29,8,23,0,44,15479,45,1,9
4332,3057,-1,2021-03-01,35,1,1,0,0,3,43,3,25,1,25,54712,39,7,9
2156,1559,12,2021-08-01,33,1,1,0,0,26,24,8,26,3,38,50988,23,2,9
2499,1779,26,2018-02-01,34,2,0,0,2,4,7,4,21,0,36,13044,50,0,9


In [27]:
df_agg.shape

(4868, 18)

In [28]:
df_agg.drop_duplicates(subset=["user_ID", "relative_renewal_date"], inplace=True)
df_agg.shape

(2437, 18)

### Get member transfer chapters' (the one they joined after transfer)

In [29]:
df_agg.loc[df_agg["chapter_ID"] == -1]

,user_ID,chapter_ID,relative_renewal_date,P,A,L,M,S,RGI,RGO,RRI,RRO,V,1-2-1,TYFCB,CEU,year_of_membership,control_count
1274,969,-1,2017-04-01,36,0,0,0,2,12,18,1,8,10,63,24515,69,0,10
1781,1324,-1,2020-02-01,22,6,0,0,2,11,17,2,30,0,42,134932,23,0,9
1945,1427,-1,2017-12-01,31,1,0,0,7,15,33,3,38,6,74,5257,17,1,11
4101,2915,-1,2020-06-01,35,1,0,0,2,5,44,17,14,7,102,41144,50,1,10
4258,3024,-1,2019-03-01,34,2,4,0,1,2,25,10,20,13,24,5900,182,2,10
4305,3047,-1,2021-05-01,30,2,0,3,0,9,41,4,9,8,32,28134,24,6,10


In [30]:
df_agg.loc[df_agg["chapter_ID"] == -1, "relative_renewal_date"] - pd.DateOffset(months=5)

1274   2016-11-01
1781   2019-09-01
1945   2017-07-01
4101   2020-01-01
4258   2018-10-01
4305   2020-12-01
Name: relative_renewal_date, dtype: datetime64[ns]

In [31]:
df_agg["chapter_transfer"] = 0
df_agg.loc[df_agg["chapter_ID"] == -1, "chapter_transfer"] = 1

In [32]:
for index, row in df_agg.loc[df_agg["chapter_ID"] == -1].iterrows():
    get_date = row["relative_renewal_date"] - pd.DateOffset(months=5)
    chapter_ID = df_master_clean.loc[(df_master_clean["user_ID"] == row["user_ID"]) & (df_master_clean["palms_date"] == get_date), "chapter_ID"]
    df_agg.loc[index, "chapter_ID"] = chapter_ID.values[0]

## Feature Engineering
### 1. Chapter size

In [33]:
df_master_clean.sample(10, random_state=13)
df_chapter_count = df_master_clean.groupby(["palms_date", "chapter_ID"])[["user_ID"]].count()
df_chapter_count.reset_index(level=["palms_date", "chapter_ID"], inplace=True)
df_chapter_count.rename({"user_ID": "chapter_size"}, axis=1, inplace=True)
df_chapter_count.sample(10, random_state=13)

,palms_date,chapter_ID,chapter_size
1016,2020-03-01,32,36
749,2019-01-01,14,24
296,2016-11-01,11,30
1299,2021-05-01,23,23
807,2019-04-01,19,34
189,2016-04-01,9,33
448,2017-08-01,31,23
1261,2021-03-01,31,19
620,2018-06-01,12,21
1097,2020-08-01,6,31


In [34]:
# Get "temp_date" for merging new features
df_agg["temp_date"] = df_agg["relative_renewal_date"] - pd.DateOffset(months=4)
df_agg.shape

(2437, 20)

In [35]:
df_agg.loc[df_agg["temp_date"].dt.day != 1]

,user_ID,chapter_ID,relative_renewal_date,P,A,L,M,S,RGI,RGO,RRI,RRO,V,1-2-1,TYFCB,CEU,year_of_membership,control_count,chapter_transfer,temp_date


In [36]:
df_agg = df_agg.merge(df_chapter_count,
                      how="left",
                      left_on=["chapter_ID", "temp_date"],
                      right_on=["chapter_ID", "palms_date"])

df_agg.drop(["palms_date"], axis=1, inplace=True)
df_agg.shape

(2437, 21)

In [37]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2437 entries, 0 to 2436
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   user_ID                2437 non-null   int64         
 1   chapter_ID             2437 non-null   int64         
 2   relative_renewal_date  2437 non-null   datetime64[ns]
 3   P                      2437 non-null   int64         
 4   A                      2437 non-null   int64         
 5   L                      2437 non-null   int64         
 6   M                      2437 non-null   int64         
 7   S                      2437 non-null   int64         
 8   RGI                    2437 non-null   int64         
 9   RGO                    2437 non-null   int64         
 10  RRI                    2437 non-null   int64         
 11  RRO                    2437 non-null   int64         
 12  V                      2437 non-null   int64         
 13  1-2

In [38]:
df_agg.loc[df_agg["chapter_size"].isna()]

,user_ID,chapter_ID,relative_renewal_date,P,A,L,M,S,RGI,RGO,...,RRO,V,1-2-1,TYFCB,CEU,year_of_membership,control_count,chapter_transfer,temp_date,chapter_size


### 2. Chapter retention rate

In [39]:
df_dropped.head()

,user_ID,chapter_ID,reason,drop_date
0,637,9,Left Company,2021-11-03
1,1185,23,Changed Job,2021-10-22
2,1225,14,Takes Too Much Time,2021-10-13
3,2138,15,Did Not Renew - No reason Given,2021-11-01
4,1610,15,Other Reason (see notes),2021-10-12


In [40]:
df_agg.head()

,user_ID,chapter_ID,relative_renewal_date,P,A,L,M,S,RGI,RGO,...,RRO,V,1-2-1,TYFCB,CEU,year_of_membership,control_count,chapter_transfer,temp_date,chapter_size
0,0,31,2016-04-01,33,0,0,1,0,21,7,...,32,1,37,129784,45,0,9,0,2015-12-01,32
1,2,18,2017-05-01,33,2,2,0,0,6,17,...,22,1,26,9285,1,0,9,0,2017-01-01,34
2,2,18,2018-05-01,35,1,0,0,2,6,24,...,36,1,20,7263,10,1,9,0,2018-01-01,21
3,2,18,2019-05-01,36,0,0,0,1,9,8,...,19,3,28,1860,31,2,9,0,2019-01-01,22
4,2,18,2020-05-01,33,1,0,0,3,10,13,...,47,4,30,6668,16,3,9,0,2020-01-01,26


In [41]:
df_master_clean.loc[(df_master_clean["user_ID"] == 0) & (df_master_clean["relative_renewal_date"] == "2016-04-01")]

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,industry,sponsor_ID,join_date,renewal_date,re_joins,reason,drop_date,year_of_membership,months_to_renewal,relative_renewal_date
1290,0,31,3,0,0,0,0,0,1,0,...,"Art, Art Framing and Products",410.0,2015-04-01,2017-04-01,1,Personal Reasons,2016-09-14,0.0,12,2016-04-01
1610,0,31,4,0,0,0,0,2,1,4,...,"Art, Art Framing and Products",410.0,2015-04-01,2017-04-01,1,Personal Reasons,2016-09-14,0.0,11,2016-04-01
1958,0,31,4,0,0,0,0,4,0,3,...,"Art, Art Framing and Products",410.0,2015-04-01,2017-04-01,1,Personal Reasons,2016-09-14,0.0,10,2016-04-01
2320,0,31,4,0,0,0,0,4,0,2,...,"Art, Art Framing and Products",410.0,2015-04-01,2017-04-01,1,Personal Reasons,2016-09-14,0.0,9,2016-04-01
2681,0,31,3,0,0,1,0,1,0,4,...,"Art, Art Framing and Products",410.0,2015-04-01,2017-04-01,1,Personal Reasons,2016-09-14,0.0,8,2016-04-01
3037,0,31,5,0,0,0,0,3,1,6,...,"Art, Art Framing and Products",410.0,2015-04-01,2017-04-01,1,Personal Reasons,2016-09-14,0.0,7,2016-04-01
3388,0,31,4,0,0,0,0,1,2,3,...,"Art, Art Framing and Products",410.0,2015-04-01,2017-04-01,1,Personal Reasons,2016-09-14,0.0,6,2016-04-01
3735,0,31,3,0,0,0,0,2,2,4,...,"Art, Art Framing and Products",410.0,2015-04-01,2017-04-01,1,Personal Reasons,2016-09-14,0.0,5,2016-04-01
4081,0,31,3,0,0,0,0,4,0,7,...,"Art, Art Framing and Products",410.0,2015-04-01,2017-04-01,1,Personal Reasons,2016-09-14,0.0,4,2016-04-01
4456,0,31,4,0,0,0,0,2,0,3,...,"Art, Art Framing and Products",410.0,2015-04-01,2017-04-01,1,Personal Reasons,2016-09-14,0.0,3,2016-04-01


In [42]:
df_agg["drop_range_A"] = df_agg["relative_renewal_date"] - pd.DateOffset(years=1)
df_agg["drop_range_B"] = df_agg["relative_renewal_date"] - pd.DateOffset(months=3)
df_agg.head()

,user_ID,chapter_ID,relative_renewal_date,P,A,L,M,S,RGI,RGO,...,1-2-1,TYFCB,CEU,year_of_membership,control_count,chapter_transfer,temp_date,chapter_size,drop_range_A,drop_range_B
0,0,31,2016-04-01,33,0,0,1,0,21,7,...,37,129784,45,0,9,0,2015-12-01,32,2015-04-01,2016-01-01
1,2,18,2017-05-01,33,2,2,0,0,6,17,...,26,9285,1,0,9,0,2017-01-01,34,2016-05-01,2017-02-01
2,2,18,2018-05-01,35,1,0,0,2,6,24,...,20,7263,10,1,9,0,2018-01-01,21,2017-05-01,2018-02-01
3,2,18,2019-05-01,36,0,0,0,1,9,8,...,28,1860,31,2,9,0,2019-01-01,22,2018-05-01,2019-02-01
4,2,18,2020-05-01,33,1,0,0,3,10,13,...,30,6668,16,3,9,0,2020-01-01,26,2019-05-01,2020-02-01


In [43]:
df_agg = df_agg.merge(df_chapter_count,
                      how="left",
                      left_on=["chapter_ID", "drop_range_A"],
                      right_on=["chapter_ID", "palms_date"])

df_agg.drop(["palms_date"], axis=1, inplace=True)
df_agg.head()

,user_ID,chapter_ID,relative_renewal_date,P,A,L,M,S,RGI,RGO,...,TYFCB,CEU,year_of_membership,control_count,chapter_transfer,temp_date,chapter_size_x,drop_range_A,drop_range_B,chapter_size_y
0,0,31,2016-04-01,33,0,0,1,0,21,7,...,129784,45,0,9,0,2015-12-01,32,2015-04-01,2016-01-01,20
1,2,18,2017-05-01,33,2,2,0,0,6,17,...,9285,1,0,9,0,2017-01-01,34,2016-05-01,2017-02-01,32
2,2,18,2018-05-01,35,1,0,0,2,6,24,...,7263,10,1,9,0,2018-01-01,21,2017-05-01,2018-02-01,33
3,2,18,2019-05-01,36,0,0,0,1,9,8,...,1860,31,2,9,0,2019-01-01,22,2018-05-01,2019-02-01,23
4,2,18,2020-05-01,33,1,0,0,3,10,13,...,6668,16,3,9,0,2020-01-01,26,2019-05-01,2020-02-01,25


In [44]:
df_agg["chapter_members_dropped"] = -1

for index, row in df_agg.iterrows():
    cond1 = (df_dropped["drop_date"] >= row["drop_range_A"]) & (df_dropped["drop_date"] <= row["drop_range_B"])
    cond2 = df_dropped["chapter_ID"] == row["chapter_ID"]
    df_agg.loc[index, "chapter_members_dropped"] = df_dropped.loc[cond1 & cond2, "user_ID"].count()

In [45]:
df_agg["chapter_retention_rate"] = (df_agg["chapter_size_y"] - df_agg["chapter_members_dropped"]) / df_agg["chapter_size_y"]
df_agg.head()

,user_ID,chapter_ID,relative_renewal_date,P,A,L,M,S,RGI,RGO,...,year_of_membership,control_count,chapter_transfer,temp_date,chapter_size_x,drop_range_A,drop_range_B,chapter_size_y,chapter_members_dropped,chapter_retention_rate
0,0,31,2016-04-01,33,0,0,1,0,21,7,...,0,9,0,2015-12-01,32,2015-04-01,2016-01-01,20,13,0.350000
1,2,18,2017-05-01,33,2,2,0,0,6,17,...,0,9,0,2017-01-01,34,2016-05-01,2017-02-01,32,9,0.718750
2,2,18,2018-05-01,35,1,0,0,2,6,24,...,1,9,0,2018-01-01,21,2017-05-01,2018-02-01,33,19,0.424242
3,2,18,2019-05-01,36,0,0,0,1,9,8,...,2,9,0,2019-01-01,22,2018-05-01,2019-02-01,23,6,0.739130
4,2,18,2020-05-01,33,1,0,0,3,10,13,...,3,9,0,2020-01-01,26,2019-05-01,2020-02-01,25,6,0.760000


In [46]:
df_agg.describe()

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,1-2-1,TYFCB,CEU,year_of_membership,control_count,chapter_transfer,chapter_size_x,chapter_size_y,chapter_members_dropped,chapter_retention_rate
count,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,...,2437.000000,2.437000e+03,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000
mean,1590.583504,17.910546,33.137464,1.574887,0.381206,0.485433,1.534263,10.384899,25.045958,10.186705,...,43.826426,2.856506e+04,42.176446,1.745589,9.009438,0.002462,29.258925,29.297907,9.972918,0.660308
std,881.489816,8.374753,3.657266,1.883467,1.104711,1.613359,1.585856,7.574072,19.691617,11.588881,...,23.833436,7.629809e+04,43.208587,2.375666,0.112413,0.049568,6.655356,6.658813,4.580614,0.144128
min,0.000000,2.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,0.000000e+00,0.000000,0.000000,9.000000,0.000000,10.000000,10.000000,0.000000,0.043478
25%,851.000000,10.000000,31.000000,0.000000,0.000000,0.000000,0.000000,5.000000,13.000000,3.000000,...,29.000000,5.314000e+03,20.000000,0.000000,9.000000,0.000000,24.000000,24.000000,7.000000,0.576923
50%,1607.000000,18.000000,34.000000,1.000000,0.000000,0.000000,1.000000,9.000000,21.000000,7.000000,...,39.000000,1.251800e+04,36.000000,1.000000,9.000000,0.000000,30.000000,30.000000,9.000000,0.676471
75%,2345.000000,25.000000,36.000000,2.000000,0.000000,0.000000,2.000000,14.000000,32.000000,13.000000,...,53.000000,2.804700e+04,51.000000,3.000000,9.000000,0.000000,34.000000,34.000000,13.000000,0.750000
max,3080.000000,32.000000,40.000000,22.000000,15.000000,22.000000,11.000000,62.000000,224.000000,126.000000,...,276.000000,1.467394e+06,829.000000,22.000000,11.000000,1.000000,44.000000,46.000000,27.000000,1.000000


### 3. Chapter growth rate

In [47]:
df_agg["chapter_growth_rate"] = -1

for index, row in df_agg.iterrows():
    cond1 = (df_database["join_date"] >= row["drop_range_A"]) & (df_database["join_date"] <= row["drop_range_B"])
    cond2 = df_database["chapter_ID"] == row["chapter_ID"]
    df_agg.loc[index, "chapter_growth_rate"] = df_database.loc[cond1 & cond2, "user_ID"].count()

In [48]:
df_agg["chapter_growth_rate"] = (df_agg["chapter_size_y"] - df_agg["chapter_members_dropped"] + df_agg["chapter_growth_rate"]) / df_agg["chapter_size_y"]
df_agg.head()

,user_ID,chapter_ID,relative_renewal_date,P,A,L,M,S,RGI,RGO,...,control_count,chapter_transfer,temp_date,chapter_size_x,drop_range_A,drop_range_B,chapter_size_y,chapter_members_dropped,chapter_retention_rate,chapter_growth_rate
0,0,31,2016-04-01,33,0,0,1,0,21,7,...,9,0,2015-12-01,32,2015-04-01,2016-01-01,20,13,0.350000,1.400000
1,2,18,2017-05-01,33,2,2,0,0,6,17,...,9,0,2017-01-01,34,2016-05-01,2017-02-01,32,9,0.718750,1.093750
2,2,18,2018-05-01,35,1,0,0,2,6,24,...,9,0,2018-01-01,21,2017-05-01,2018-02-01,33,19,0.424242,0.636364
3,2,18,2019-05-01,36,0,0,0,1,9,8,...,9,0,2019-01-01,22,2018-05-01,2019-02-01,23,6,0.739130,1.217391
4,2,18,2020-05-01,33,1,0,0,3,10,13,...,9,0,2020-01-01,26,2019-05-01,2020-02-01,25,6,0.760000,1.120000


In [49]:
df_agg.drop(["drop_range_A", "drop_range_B", "chapter_size_y", "chapter_members_dropped"], axis=1, inplace=True)
df_agg.rename({"chapter_size_x": "chapter_size"}, axis=1, inplace=True)

In [50]:
df_agg.describe()

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,V,1-2-1,TYFCB,CEU,year_of_membership,control_count,chapter_transfer,chapter_size,chapter_retention_rate,chapter_growth_rate
count,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,...,2437.000000,2437.000000,2.437000e+03,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000,2437.000000
mean,1590.583504,17.910546,33.137464,1.574887,0.381206,0.485433,1.534263,10.384899,25.045958,10.186705,...,3.851046,43.826426,2.856506e+04,42.176446,1.745589,9.009438,0.002462,29.258925,0.660308,1.182979
std,881.489816,8.374753,3.657266,1.883467,1.104711,1.613359,1.585856,7.574072,19.691617,11.588881,...,3.970278,23.833436,7.629809e+04,43.208587,2.375666,0.112413,0.049568,6.655356,0.144128,0.345921
min,0.000000,2.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,4.000000,0.000000e+00,0.000000,0.000000,9.000000,0.000000,10.000000,0.043478,0.575758
25%,851.000000,10.000000,31.000000,0.000000,0.000000,0.000000,0.000000,5.000000,13.000000,3.000000,...,1.000000,29.000000,5.314000e+03,20.000000,0.000000,9.000000,0.000000,24.000000,0.576923,1.000000
50%,1607.000000,18.000000,34.000000,1.000000,0.000000,0.000000,1.000000,9.000000,21.000000,7.000000,...,3.000000,39.000000,1.251800e+04,36.000000,1.000000,9.000000,0.000000,30.000000,0.676471,1.107143
75%,2345.000000,25.000000,36.000000,2.000000,0.000000,0.000000,2.000000,14.000000,32.000000,13.000000,...,5.000000,53.000000,2.804700e+04,51.000000,3.000000,9.000000,0.000000,34.000000,0.750000,1.269231
max,3080.000000,32.000000,40.000000,22.000000,15.000000,22.000000,11.000000,62.000000,224.000000,126.000000,...,36.000000,276.000000,1.467394e+06,829.000000,22.000000,11.000000,1.000000,44.000000,1.000000,2.807692


### 4. Seat popularity rate

In [51]:
df_master_clean.head()

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,industry,sponsor_ID,join_date,renewal_date,re_joins,reason,drop_date,year_of_membership,months_to_renewal,relative_renewal_date
0,447,9,4,0,0,0,0,1,1,0,...,"Finance & Insurance, Financial Investments",NaN,1998-05-01,2022-07-01,1,NaN,NaT,16.0,6,2015-07-01
1,1366,9,4,0,0,0,0,4,0,0,...,"Advertising & Marketing, Sign Company",NaN,2002-09-01,2022-06-01,1,NaN,NaT,12.0,5,2015-06-01
2,1197,9,4,0,0,0,0,2,3,0,...,"Coach, Business Coach",NaN,2006-11-01,2018-02-01,1,No Reason Entered,2018-01-11,8.0,1,2015-02-01
3,1035,9,4,0,0,0,0,0,7,0,...,"Insurance, Property & Casualty Insurance",1366.0,2007-07-01,2015-05-01,1,No Reason Entered,2015-06-04,7.0,4,2015-05-01
4,1536,23,3,0,0,0,1,0,3,0,...,"Real Estate Services, Residential Real Estate ...",NaN,2007-07-01,2021-04-01,2,Member Transferred BNI Chapters,2020-12-18,7.0,3,2015-04-01


In [52]:
df_agg = df_agg.merge(df_master_clean[["user_ID", "chapter_ID", "industry"]].drop_duplicates(),
                      how="left",
                      on=["user_ID", "chapter_ID"])
df_agg.shape

(2437, 24)

In [53]:
df_chapter_count = df_master_clean.groupby("palms_date")[["chapter_ID"]].nunique()
df_chapter_count.rename({"chapter_ID": "chapter_count"}, axis=1, inplace=True)
df_chapter_count.head()

,chapter_count
palms_date,
2015-01-01,12
2015-02-01,12
2015-03-01,12
2015-04-01,12
2015-05-01,12


In [54]:
df_agg = df_agg.merge(df_chapter_count, how="left", left_on="temp_date", right_index=True)
df_agg.shape

(2437, 25)

In [55]:
df_master_clean.head()

,user_ID,chapter_ID,P,A,L,M,S,RGI,RGO,RRI,...,industry,sponsor_ID,join_date,renewal_date,re_joins,reason,drop_date,year_of_membership,months_to_renewal,relative_renewal_date
0,447,9,4,0,0,0,0,1,1,0,...,"Finance & Insurance, Financial Investments",NaN,1998-05-01,2022-07-01,1,NaN,NaT,16.0,6,2015-07-01
1,1366,9,4,0,0,0,0,4,0,0,...,"Advertising & Marketing, Sign Company",NaN,2002-09-01,2022-06-01,1,NaN,NaT,12.0,5,2015-06-01
2,1197,9,4,0,0,0,0,2,3,0,...,"Coach, Business Coach",NaN,2006-11-01,2018-02-01,1,No Reason Entered,2018-01-11,8.0,1,2015-02-01
3,1035,9,4,0,0,0,0,0,7,0,...,"Insurance, Property & Casualty Insurance",1366.0,2007-07-01,2015-05-01,1,No Reason Entered,2015-06-04,7.0,4,2015-05-01
4,1536,23,3,0,0,0,1,0,3,0,...,"Real Estate Services, Residential Real Estate ...",NaN,2007-07-01,2021-04-01,2,Member Transferred BNI Chapters,2020-12-18,7.0,3,2015-04-01


In [56]:
df_agg["seat_popularity_rate"] = -1

for index, row in df_agg.iterrows():
    cond1 = df_master_clean["industry"] == row["industry"]
    cond2 = df_master_clean["palms_date"] == row["temp_date"]
    df_agg.loc[index, "seat_popularity_rate"] = df_master_clean.loc[cond1 & cond2, "user_ID"].count()

df_agg["seat_popularity_rate"] = df_agg["seat_popularity_rate"] / df_agg["chapter_count"]
df_agg.sample(10, random_state=23)

,user_ID,chapter_ID,relative_renewal_date,P,A,L,M,S,RGI,RGO,...,year_of_membership,control_count,chapter_transfer,temp_date,chapter_size,chapter_retention_rate,chapter_growth_rate,industry,chapter_count,seat_popularity_rate
1035,1394,11,2020-03-01,38,0,0,0,1,19,34,...,6,9,0,2019-11-01,21,0.608696,1.043478,"Training & Coaching, Business Training/Coach",18,0.277778
2,2,18,2018-05-01,35,1,0,0,2,6,24,...,1,9,0,2018-01-01,21,0.424242,0.636364,"Health & Wellness, Chiropractor",19,0.210526
505,701,15,2019-05-01,29,5,0,0,4,7,44,...,1,9,0,2019-01-01,31,0.620690,1.275862,"Trades, Heating & A/C",19,0.210526
925,1272,15,2021-10-01,36,1,0,0,0,22,56,...,2,9,0,2021-06-01,38,0.828571,1.142857,"Construction, Renovations/Remodeling",22,0.227273
2085,2695,6,2020-07-01,32,3,0,0,3,21,19,...,0,9,0,2020-03-01,33,0.650000,1.100000,"Finance & Insurance, Property & Casualty Insur...",20,0.450000
2222,2863,11,2019-04-01,36,0,0,1,0,10,2,...,0,9,0,2018-12-01,22,0.580645,0.774194,"Advertising & Marketing, Printer",19,0.157895
1308,1709,31,2018-12-01,34,1,0,0,2,5,43,...,3,9,0,2018-08-01,23,0.695652,1.130435,"Real Estate, Property Management",18,0.166667
1733,2226,10,2017-10-01,33,2,1,0,1,7,6,...,0,9,0,2017-06-01,20,0.636364,0.954545,"Landscaping, Landscaping: Design & Construction",17,0.117647
53,90,17,2016-09-01,38,0,0,0,0,34,15,...,2,9,0,2016-05-01,34,0.634146,0.853659,"z(Archived Duplicate) Alternative Medicine, Nu...",15,0.066667
2049,2642,18,2017-03-01,31,2,0,4,3,5,15,...,1,9,0,2016-11-01,34,0.700000,1.266667,"Insurance, Property & Casualty Insurance",15,0.466667


In [57]:
df_agg.shape

(2437, 26)

In [58]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2437 entries, 0 to 2436
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   user_ID                 2437 non-null   int64         
 1   chapter_ID              2437 non-null   int64         
 2   relative_renewal_date   2437 non-null   datetime64[ns]
 3   P                       2437 non-null   int64         
 4   A                       2437 non-null   int64         
 5   L                       2437 non-null   int64         
 6   M                       2437 non-null   int64         
 7   S                       2437 non-null   int64         
 8   RGI                     2437 non-null   int64         
 9   RGO                     2437 non-null   int64         
 10  RRI                     2437 non-null   int64         
 11  RRO                     2437 non-null   int64         
 12  V                       2437 non-null   int64   

In [59]:
df_agg.loc[df_agg["industry"].isna()].shape

(3, 26)

In [60]:
# Fixing "industry" for the users which are lacking it
for index, row in df_agg.loc[df_agg["industry"].isna()].iterrows():
    user_industry = df_database.loc[df_database["user_ID"] == row["user_ID"], "industry"].values[-1]
    df_agg.loc[index, "industry"] = user_industry
    cond1 = df_master_clean["industry"] == user_industry
    cond2 = df_master_clean["palms_date"] == row["temp_date"]
    df_agg.loc[index, "seat_popularity_rate"] = df_master_clean.loc[cond1 & cond2, "user_ID"].count()
    df_agg.loc[index, "seat_popularity_rate"] = df_agg.loc[index, "seat_popularity_rate"] / df_agg.loc[index, "chapter_count"]

In [61]:
df_agg.sample(10, random_state=23)

,user_ID,chapter_ID,relative_renewal_date,P,A,L,M,S,RGI,RGO,...,year_of_membership,control_count,chapter_transfer,temp_date,chapter_size,chapter_retention_rate,chapter_growth_rate,industry,chapter_count,seat_popularity_rate
1035,1394,11,2020-03-01,38,0,0,0,1,19,34,...,6,9,0,2019-11-01,21,0.608696,1.043478,"Training & Coaching, Business Training/Coach",18,0.277778
2,2,18,2018-05-01,35,1,0,0,2,6,24,...,1,9,0,2018-01-01,21,0.424242,0.636364,"Health & Wellness, Chiropractor",19,0.210526
505,701,15,2019-05-01,29,5,0,0,4,7,44,...,1,9,0,2019-01-01,31,0.620690,1.275862,"Trades, Heating & A/C",19,0.210526
925,1272,15,2021-10-01,36,1,0,0,0,22,56,...,2,9,0,2021-06-01,38,0.828571,1.142857,"Construction, Renovations/Remodeling",22,0.227273
2085,2695,6,2020-07-01,32,3,0,0,3,21,19,...,0,9,0,2020-03-01,33,0.650000,1.100000,"Finance & Insurance, Property & Casualty Insur...",20,0.450000
2222,2863,11,2019-04-01,36,0,0,1,0,10,2,...,0,9,0,2018-12-01,22,0.580645,0.774194,"Advertising & Marketing, Printer",19,0.157895
1308,1709,31,2018-12-01,34,1,0,0,2,5,43,...,3,9,0,2018-08-01,23,0.695652,1.130435,"Real Estate, Property Management",18,0.166667
1733,2226,10,2017-10-01,33,2,1,0,1,7,6,...,0,9,0,2017-06-01,20,0.636364,0.954545,"Landscaping, Landscaping: Design & Construction",17,0.117647
53,90,17,2016-09-01,38,0,0,0,0,34,15,...,2,9,0,2016-05-01,34,0.634146,0.853659,"z(Archived Duplicate) Alternative Medicine, Nu...",15,0.066667
2049,2642,18,2017-03-01,31,2,0,4,3,5,15,...,1,9,0,2016-11-01,34,0.700000,1.266667,"Insurance, Property & Casualty Insurance",15,0.466667


In [62]:
df_agg.drop(["chapter_count"], axis=1, inplace=True)

## Label records
1. Join ```df_agg``` with ```df_dropped```
2. Double-check if the ```drop_date``` is alligned with the last record in ```df_master_cleaned```

then label accordingly.

In [63]:
df_dropped.head()

,user_ID,chapter_ID,reason,drop_date
0,637,9,Left Company,2021-11-03
1,1185,23,Changed Job,2021-10-22
2,1225,14,Takes Too Much Time,2021-10-13
3,2138,15,Did Not Renew - No reason Given,2021-11-01
4,1610,15,Other Reason (see notes),2021-10-12


In [64]:
df_final = df_agg.merge(df_dropped, how="left", on=["user_ID", "chapter_ID"])
df_final.shape

(2579, 27)

In [65]:
df_final.drop("reason", axis=1, inplace=True)
df_final.drop_duplicates(df_final.columns[:16], keep="last", inplace=True)
df_final.shape

(2437, 26)

In [66]:
final_month = df_master_clean.groupby(["user_ID", "chapter_ID"])[["palms_date"]].max()
final_month.reset_index(["user_ID", "chapter_ID"], inplace=True)
final_month.rename({"palms_date": "final_palms_date"}, axis=1, inplace=True)
final_month.head()

,user_ID,chapter_ID,final_palms_date
0,0,31,2016-09-01
1,2,18,2021-10-01
2,3,11,2017-09-01
3,4,18,2021-10-01
4,6,4,2017-11-01


In [67]:
df_final = df_final.merge(final_month, how="left", on=["user_ID", "chapter_ID"])
df_final.drop_duplicates(df_final.columns[:16], keep="last", inplace=True)
df_final.sample(10, random_state=13)

,user_ID,chapter_ID,relative_renewal_date,P,A,L,M,S,RGI,RGO,...,control_count,chapter_transfer,temp_date,chapter_size,chapter_retention_rate,chapter_growth_rate,industry,seat_popularity_rate,drop_date,final_palms_date
405,573,15,2019-05-01,38,0,0,0,0,13,55,...,9,0,2019-01-01,31,0.620690,1.275862,"Trades, Plumber",0.526316,2019-04-17,2019-04-01
2187,2825,25,2017-04-01,29,3,4,0,2,7,29,...,9,0,2016-12-01,22,0.640000,0.920000,"Financial, Investment Advisor",0.666667,2019-04-03,2019-04-01
557,766,9,2020-02-01,38,1,0,0,0,9,29,...,9,0,2019-10-01,34,0.641026,0.871795,"Computer & Programming, IT & Networks",0.277778,2021-06-16,2021-06-01
99,142,24,2021-09-01,37,0,0,0,0,3,41,...,9,0,2021-05-01,19,0.772727,0.909091,"Health & Wellness, Pharmacist",0.045455,NaT,2021-10-01
722,1000,11,2021-05-01,37,0,0,0,0,21,16,...,9,0,2021-01-01,25,0.850000,1.250000,"Car & Motorcycle, Auto/Car Repair",0.363636,NaT,2021-10-01
1621,2086,9,2022-01-01,36,1,0,0,0,2,7,...,9,0,2021-09-01,38,0.764706,1.176471,"Computer & Programming, Computer Software",0.090909,NaT,2021-10-01
764,1060,23,2017-09-01,26,7,1,0,3,2,25,...,9,0,2017-05-01,35,0.694444,1.027778,"Security, Security Services",0.294118,2017-08-07,2017-08-01
517,712,23,2020-10-01,34,1,0,0,1,6,4,...,9,0,2020-06-01,35,0.729730,1.081081,"Finance & Insurance, Financial Advisor",0.150000,2021-05-01,2021-05-01
177,259,32,2018-11-01,35,2,0,0,0,7,14,...,9,0,2018-07-01,34,0.750000,0.972222,"Mortgage, Residential Mortgage",0.388889,2020-10-31,2020-11-01
1619,2075,9,2016-11-01,32,1,2,0,2,11,34,...,9,0,2016-07-01,33,0.766667,1.200000,"Employment, Recruiter",0.066667,2016-11-01,2016-11-01


In [68]:
# Remove data which cannot be labelled
df_final = df_final.loc[df_final["relative_renewal_date"] <= "2021-09-01"]
df_final.shape

(2217, 27)

In [69]:
df_final[["user_ID", "chapter_ID", "relative_renewal_date", "drop_date", "final_palms_date"]].sample(10, random_state=13)

,user_ID,chapter_ID,relative_renewal_date,drop_date,final_palms_date
491,680,12,2020-10-01,NaT,2021-10-01
1011,1366,9,2021-06-01,NaT,2021-10-01
403,571,32,2017-06-01,2017-06-01,2017-06-01
2351,3005,32,2016-09-01,NaT,2021-10-01
968,1335,23,2018-12-01,2020-05-15,2020-06-01
1636,2109,14,2021-05-01,2021-03-29,2021-03-01
1792,2299,11,2021-01-01,NaT,2021-10-01
247,362,18,2016-12-01,2017-08-31,2017-09-01
2364,3015,9,2017-05-01,NaT,NaT
984,1352,9,2016-01-01,NaT,2021-10-01


In [70]:
df_final["date_diff"] = df_final["final_palms_date"] - df_final["drop_date"]
df_final.head()

,user_ID,chapter_ID,relative_renewal_date,P,A,L,M,S,RGI,RGO,...,chapter_transfer,temp_date,chapter_size,chapter_retention_rate,chapter_growth_rate,industry,seat_popularity_rate,drop_date,final_palms_date,date_diff
0,0,31,2016-04-01,33,0,0,1,0,21,7,...,0,2015-12-01,32,0.350000,1.400000,"Art, Art Framing and Products",0.076923,2016-09-14,2016-09-01,-13 days
1,2,18,2017-05-01,33,2,2,0,0,6,17,...,0,2017-01-01,34,0.718750,1.093750,"Health & Wellness, Chiropractor",0.266667,NaT,2021-10-01,NaT
2,2,18,2018-05-01,35,1,0,0,2,6,24,...,0,2018-01-01,21,0.424242,0.636364,"Health & Wellness, Chiropractor",0.210526,NaT,2021-10-01,NaT
3,2,18,2019-05-01,36,0,0,0,1,9,8,...,0,2019-01-01,22,0.739130,1.217391,"Health & Wellness, Chiropractor",0.263158,NaT,2021-10-01,NaT
4,2,18,2020-05-01,33,1,0,0,3,10,13,...,0,2020-01-01,26,0.760000,1.120000,"Health & Wellness, Chiropractor",0.277778,NaT,2021-10-01,NaT


In [71]:
df_final.loc[df_final["date_diff"] > pd.Timedelta(days=31)].shape

(159, 28)

In [72]:
df_final.loc[df_final["drop_date"].isna()].shape

(843, 28)

In [73]:
df_final.loc[df_final["final_palms_date"].isna()].shape

(3, 28)

In [74]:
df_final.loc[df_final["final_palms_date"].isna()]

,user_ID,chapter_ID,relative_renewal_date,P,A,L,M,S,RGI,RGO,...,chapter_transfer,temp_date,chapter_size,chapter_retention_rate,chapter_growth_rate,industry,seat_popularity_rate,drop_date,final_palms_date,date_diff
1063,1420,19,2019-11-01,33,2,1,0,1,2,10,...,0,2019-07-01,37,0.702703,1.432432,"Consulting, Business Consultant - Organization...",0.055556,NaT,NaT,NaT
2280,2909,19,2018-06-01,35,1,0,0,1,6,15,...,0,2018-02-01,30,0.612903,1.064516,"Trades, Electrical Contractor",0.263158,NaT,NaT,NaT
2364,3015,9,2017-05-01,34,1,0,0,3,16,25,...,0,2017-01-01,35,0.741935,1.064516,"Food & Beverage, Restaurant",0.066667,NaT,NaT,NaT


In [75]:
for index, row in df_final.loc[df_final["final_palms_date"].isna()].iterrows():
    df_final.loc[index, "final_palms_date"] = df_master_clean.loc[df_master_clean["user_ID"] == row["user_ID"], "palms_date"].max()

In [76]:
df_final.loc[df_final["final_palms_date"].isna()].shape

(0, 28)

In [77]:
df_final.loc[df_final["date_diff"] > pd.Timedelta(days=31)].sample(5, random_state=13)

,user_ID,chapter_ID,relative_renewal_date,P,A,L,M,S,RGI,RGO,...,chapter_transfer,temp_date,chapter_size,chapter_retention_rate,chapter_growth_rate,industry,seat_popularity_rate,drop_date,final_palms_date,date_diff
875,1198,18,2017-02-01,32,1,0,2,1,2,16,...,0,2016-10-01,34,0.653846,1.269231,"Automotive, Automotive Sales & Leasing",0.066667,2016-03-11,2018-01-01,661 days
1101,1464,11,2017-02-01,36,3,0,0,0,7,18,...,0,2016-10-01,28,0.714286,1.571429,"Trades, Painter",0.466667,2018-02-01,2018-09-01,212 days
1956,2506,12,2020-07-01,39,1,0,0,0,2,40,...,0,2020-03-01,28,0.566667,1.133333,"Real Estate, Real Estate Sales Representative ...",0.450000,2019-08-01,2021-10-01,792 days
2383,3037,24,2021-06-01,24,10,0,0,0,7,2,...,0,2021-02-01,20,0.695652,0.913043,"Trades, Roofing",0.045455,2018-04-04,2021-02-01,1034 days
1242,1631,23,2020-05-01,32,2,0,0,4,12,10,...,0,2020-01-01,37,0.735294,1.205882,"Insurance, Group Benefits Consultant",0.055556,2016-02-02,2020-02-01,1460 days


It looks as though ```final_palms_date``` (final available PALMS data record per user) is more reliable than the ```drop_date``` and so moving forward the former will be used to determine labels:

- "1" if a member was dropped 
- "0" if the member will be renewing

In [78]:
df_final["wont_renew"] = 0
cond = df_final["relative_renewal_date"] + pd.DateOffset(months=1) >= df_final["final_palms_date"]
df_final.loc[cond, "wont_renew"] = 1
df_final.loc[cond, "wont_renew"].shape

(679,)

In [79]:
df_final[["user_ID", "chapter_ID", "relative_renewal_date", "final_palms_date", "wont_renew"]].sample(10, random_state=13)

,user_ID,chapter_ID,relative_renewal_date,final_palms_date,wont_renew
491,680,12,2020-10-01,2021-10-01,0
1011,1366,9,2021-06-01,2021-10-01,0
403,571,32,2017-06-01,2017-06-01,1
2351,3005,32,2016-09-01,2021-10-01,0
968,1335,23,2018-12-01,2020-06-01,0
1636,2109,14,2021-05-01,2021-03-01,1
1792,2299,11,2021-01-01,2021-10-01,0
247,362,18,2016-12-01,2017-09-01,0
2364,3015,9,2017-05-01,2021-10-01,0
984,1352,9,2016-01-01,2021-10-01,0


In [80]:
df_final.columns

Index(['user_ID', 'chapter_ID', 'relative_renewal_date', 'P', 'A', 'L', 'M',
       'S', 'RGI', 'RGO', 'RRI', 'RRO', 'V', '1-2-1', 'TYFCB', 'CEU',
       'year_of_membership', 'control_count', 'chapter_transfer', 'temp_date',
       'chapter_size', 'chapter_retention_rate', 'chapter_growth_rate',
       'industry', 'seat_popularity_rate', 'drop_date', 'final_palms_date',
       'date_diff', 'wont_renew'],
      dtype='object')

In [81]:
df_final.to_csv("data/df_final.csv")